In [12]:
from langchain_ollama import ChatOllama

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (RunnableSequence, RunnableLambda,
                                      RunnablePassthrough,RunnableParallel)



In [13]:
def word_count(text):
    return len(text.split())


prompt=PromptTemplate(
    template='write a joke about {topic}',
    input_variables=['topic']
)
model=ChatOllama(model="gemma3:270m")
parser=StrOutputParser()

In [4]:
joke_gen_chain=RunnableSequence(prompt,model,parser)

parallel_chain=RunnableParallel(({
    'joke':RunnablePassthrough(),
    'word_count':RunnableLambda(word_count)
}))

final_chain=RunnableSequence(joke_gen_chain,parallel_chain)

In [5]:
result=final_chain.invoke({'topic':'AI'})

In [6]:
final_result='''{}\nword_count-{}'''.format(result['joke'],result['word_count'])

In [7]:
print(final_result)

Why did the AI cross the road? 

To prove it could do anything!

word_count-13


In [8]:
from pydantic import BaseModel,Field
from typing import Literal
from langchain_core.output_parsers import PydanticOutputParser

class Feedback(BaseModel):
    sentiment:Literal['Positive','Negative','Neutral']=Field(description='Give the sentiment of the Feedback')


parser2=PydanticOutputParser(pydantic_object=Feedback)

prompt1=PromptTemplate(
    template="Classify the sentiment of the following feedback:\n\n{feedback}\n\n{format_instruction}",
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)


In [9]:
model=ChatOllama(model='deepseek-r1:1.5b')
classifier_chain=prompt1|model|parser2
result=classifier_chain.invoke({'feedback':"I dont like  the product"})

In [10]:
print(result.sentiment)

Negative


## Hugging Face With Langchain

In [1]:
from transformers import AutoModelForCausalLM,AutoTokenizer,pipeline
from langchain_huggingface import HuggingFacePipeline


d:\Mlops\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Choose the model variant
model_id ="Qwen/Qwen3-0.6B"
tokenizer=AutoTokenizer.from_pretrained(model_id)
hf_model=AutoModelForCausalLM.from_pretrained(model_id)

In [3]:
pipe=pipeline(
    "text-generation",
    model=hf_model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.7
)

Device set to use cpu


In [4]:
model_hf=HuggingFacePipeline(pipeline=pipe)

In [7]:
prompt=PromptTemplate(
    template="write a  joke on :{topic}",
    input_variables=['topic']
)

In [14]:
chain = prompt | model_hf | parser

result = chain.invoke({"topic": " joke about trump"})
print(result)


write a joke about  joke about trump

I am a joke writer. Let me try to come up with a joke. The joke is about a person who is in a situation where they have to make a choice between two options. The options are either to take a risk and have a chance at something good, or to take a risk and have a chance at something bad. The person is in a situation where they have to choose between the two options, and the answer is that the person would choose the first option. The joke should


In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('top news in india today')

print(results)

In [ ]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

In [ ]:
from langchain_core.tools import tool
# Step 2 - add type hints
# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b